# The Three-Body Problem - 2D #

by Julián Ramón Marrades Furquet <br>
Department of Data Science and Knowledge Engineering <br>
Maastricht University

## What is the Three-Body Problem?##
According to Wikipedia, in physics and classical mechanics, the three-body problem is the problem of taking the initial positions and velocities (or momenta) of three point masses and solving for their subsequent motion according to Newton's laws of motion and Newton's law of universal gravitation. <br>
The three-body problem is a special case of the n-body problem. Unlike two-body problems, no closed-form solution exists for all sets of initial conditions, and numerical methods are generally required.

## Formal definition ##
Our startpoint is the **Newton's law of universal gravitation**, which states that 2 bodies with masses $m_1$ and $m_2$ attract each other with a force:

$$\vec{F} = -G \cdot \frac{m_1 \cdot m_2}{\vec{r}^{\ 2}}$$

where $\vec{r}$ is the distance between the center of their masses and $G$ is the gravitational constant, which we will take as $G=1$ for simplicity.

Now, we are interested in the acceleration which a body suffers when it is attracted by other masses, because it is the second derivative of its displacement. Therefore, we can use it to describe our system. <br>
For instance, we can state the acceleration of the body with $m_1$ when it is attracted by $m_2$ employing Newton's equations:

$$\vec{a}_1 = \frac{\vec{F}_1}{m_1} = -G \cdot \frac{m_2}{\vec{r}^{\ 2}}$$

Since we are normalizing $G$, we can simplify the equation to:

$$\vec{a}_1 = - \frac{m_2}{\vec{r}^{\ 2}}$$

If $m_1$ was part of a system of $n$ bodies, we could perform vector sum to express its overall acceleration caused by the attraction of the other $n-1$ masses:

$$\vec{a}_1 = - \frac{m_2}{\vec{r}_{12}^{\ 2}} - \frac{m_3}{\vec{r}_{13}^{\ 2}} - \dots - \frac{m_n}{\vec{r}_{1n}^{\ 2}}$$

## Building the system ##
To build our system we will perform two simple steps:
1. Obtain the modulus of the acceleration that a body suffers
2. Point it towards the correct direction (i.e. vectorize it)

#### 1. Obtaining the modulus ####

We can define the modulus of the acceleration that $m_1$ suffers when attracted by $m_2$ as:

$$a_1 = - \frac{m_2}{r_{12}^2}$$

#### 2. Vectorization of the modulus ####

Due to the negative sign on the previous equation, we need a unit vector pointing from $m_2$ to $m_1$, so that we can invert it and scale it later. We can do it as follows:

$$\vec{v} = \frac{p_1 - p_2}{\vert p_1 - p_2 \vert}$$

If this is somewhat unclear, we can also visualize it in the 2D plane:
<img src="imgs/grav_2masses.png" style="max-width:100%; width: 20%" />

Now it is time to bring the two previous equations together.

First, let's redefine $r_{12}^2$:

$$r_{12} = \vert p_1 - p_2 \vert \rightarrow r_{12}^2 = \vert p_1 - p_2 \vert ^ 2$$

Then, we bring the equations together and rearrange the terms:

$$\vec{a_1} = - \frac{m_2}{\vert p_1 - p_2 \vert^2} \cdot \frac{p_1 - p_2}{\vert p_1 - p_2 \vert} = - m_2 \frac{p_1 - p_2}{\vert p_1 - p_2 \vert^3}$$

Generalizing, if $m_1$ was attracted by more bodies:

$$\vec{a_1} = - m_2 \frac{p_1 - p_2}{\vert p_1 - p_2 \vert^3} - m_3 \frac{p_1 - p_3}{\vert p_1 - p_3 \vert^3} - \dots$$

However, we will limit ourselves to 3 masses (by now). Hence, since the acceleration is the second derivative of the displacement (or position), we can generate a system of 3 second order differential equations to describe the interactions of 3 bodies:

$$\ddot{p}_1 = - m_2 \cdot \frac{p_1 - p_2}{\vert p_1 - p_2 \vert^3} - m_3 \frac{p_1 - p_3}{\vert p_1 - p_3 \vert^3}$$

$$\ddot{p}_2 = - m_1 \cdot \frac{p_2 - p_1}{\vert p_2 - p_1 \vert^3} - m_3 \frac{p_2 - p_3}{\vert p_2 - p_3 \vert^3}$$

$$\ddot{p}_3 = - m_1 \cdot \frac{p_3 - p_1}{\vert p_3 - p_1 \vert^3} - m_2 \frac{p_3 - p_2}{\vert p_3 - p_2 \vert^3}$$

## Implementation ##

Since we are working with ODE solvers within the `scipy` library, we need to transform this into a system of 6 ordinary differential equations.

We will use $s$ as our state vector such that
$$s = [p_1, \dot{p}_1, p_2, \dot{p}_2, p_3, \dot{p}_3]$$
Hence,
$$\dot{s} = [\dot{p}_1, \ddot{p}_1, \dot{p}_2, \ddot{p}_2, \dot{p}_3, \ddot{p}_3]$$

**Note:** every element of the vectors is a vector itself, representing the $x$ and $y$ coordinates.

---
Let's start by importing everything we will need during the process

In [1]:
%matplotlib inline
import numpy as np
from scipy.integrate import odeint
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation

Then, we need to define the masses and initial conditions of the bodies

In [1]:
m1, m2, m3 = 1, 1, 1

Now, we define the function `ds(s,t)`

In [1]:
def ds(s, t):
    """This function computes the derivative of the state."""
    
    # Rename variables for convenience
    r1x, r1y, dr1x, dr1y = x[0], x[1], x[2], x[3]
    r2x, r2y, dr2x, dr2y = x[4], x[5], x[6], x[7]
    r3x, r3y, dr3x, dr3y = x[8], x[9], x[10], x[11]
    
    # Compute the second derivatives
    ddr1x = 0
    ddr1y = 0
    ddr2x = 0
    ddr2y = 0
    ddr3x = 0
    ddr3y = 0
    
    # Return the derivative
    return [dr1x, dr1y, ddr1x, ddr1y, dr2x, dr2y, ddr2x, ddr2y, dr3x, dr3y, ddr3x, ddr3y]